


          
### 阿里云ECS服务器部署Django网站详细步骤（含代码）

#### 一、服务器准备与连接

##### 1. 配置安全组规则
   - 登录阿里云控制台，进入ECS实例管理页面
   - 点击左侧菜单栏的「安全组」
   - 选择实例所在的安全组，点击「配置规则」
   - 点击「添加安全组规则」，添加以下规则：
     - 端口范围：22/22，授权对象：0.0.0.0/0（SSH连接）
     - 端口范围：80/80，授权对象：0.0.0.0/0（HTTP访问）
     - 端口范围：443/443，授权对象：0.0.0.0/0（HTTPS访问）
     - 端口范围：8000/8000，授权对象：0.0.0.0/0（Django测试）

##### 2. 远程连接服务器
   - **Windows用户**：使用PuTTY或Windows Terminal
     ```
     # 打开PuTTY，输入以下信息
     主机名/IP地址：您的公网IP
     端口：22
     连接类型：SSH
     ```
   - **macOS/Linux用户**：使用终端
     ```bash
     ssh root@您的公网IP
     ```
   - 首次连接会提示确认指纹，输入`yes`后按回车
   - 输入服务器密码（创建实例时设置的密码）

#### 二、安装必要软件

##### 1. 更新系统包
```bash
# Ubuntu系统
apt update && apt upgrade -y

# CentOS系统
yum update -y
```

##### 2. 安装Python和相关工具
```bash
# Ubuntu系统
apt install python3 python3-pip python3-venv git nginx -y

# CentOS系统
yum install python3 python3-pip git nginx -y
```

#### 三、创建Python虚拟环境

##### 1. 安装虚拟环境工具
```bash
pip3 install virtualenv virtualenvwrapper
```

##### 2. 配置虚拟环境
```bash
# 创建虚拟环境目录
mkdir -p /root/.virtualenvs

# 编辑.bashrc文件，添加以下内容
echo "export WORKON_HOME=/root/.virtualenvs" >> ~/.bashrc
echo "export VIRTUALENVWRAPPER_PYTHON=/usr/bin/python3" >> ~/.bashrc
echo "source /usr/local/bin/virtualenvwrapper.sh" >> ~/.bashrc

# 使配置生效
source ~/.bashrc
```

##### 3. 创建并激活Django项目虚拟环境
```bash
# 创建虚拟环境
mkvirtualenv django_env

# 激活虚拟环境
workon django_env
```

#### 四、上传Django项目

##### 1. 方式一：使用Git克隆（推荐）
```bash
# 安装Git（如果未安装）
apt install git -y

# 克隆项目到服务器
cd /var/www
git clone 您的项目仓库地址 django_project
```

##### 2. 方式二：使用SCP上传
```bash
# 在本地计算机的命令行中执行
scp -r /本地项目路径 root@您的公网IP:/var/www/django_project
```

#### 五、配置Django项目

##### 1. 安装项目依赖
```bash
# 进入项目目录
cd /var/www/django_project

# 激活虚拟环境（如果未激活）
workon django_env

# 安装依赖
pip install -r requirements.txt

# 安装uWSGI
pip install uwsgi
```

##### 2. 修改Django项目配置
```bash
# 编辑settings.py文件
nano mblog/settings.py
```

修改以下内容：
```python
# 允许的主机
ALLOWED_HOSTS = ['您的公网IP', '您的域名（如果有）']

# 静态文件配置
STATIC_URL = '/static/'
STATIC_ROOT = os.path.join(BASE_DIR, 'staticfiles')

# 媒体文件配置
MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')

# 数据库配置（保持SQLite3）
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```

##### 3. 迁移数据库和收集静态文件
```bash
# 迁移数据库
python manage.py migrate

# 收集静态文件
python manage.py collectstatic --noinput

# 创建超级用户（可选）
python manage.py createsuperuser
```

#### 六、配置uWSGI

##### 1. 创建uWSGI配置文件
```bash
# 进入项目目录
cd /var/www/django_project

# 创建配置文件
nano uwsgi.ini
```

添加以下内容：
```ini
[uwsgi]
# 项目目录
chdir = /var/www/django_project
# 项目的wsgi.py文件路径
wsgi-file = mblog/wsgi.py
# 虚拟环境路径
home = /root/.virtualenvs/django_env
# 进程设置
master = true
processes = 4
threads = 2
# 监听端口
socket = 127.0.0.1:8000
# 日志设置
daemonize = /var/log/uwsgi.log
# 进程文件
touch-reload = /var/www/django_project/reload
evil-reload = /var/www/django_project/reload
```

#### 七、配置Nginx

##### 1. 创建Nginx配置文件
```bash
# 创建配置文件
nano /etc/nginx/sites-available/django_project
```

添加以下内容：
```nginx
server {
    listen 80;
    server_name 您的公网IP 您的域名（如果有）;

    # 静态文件配置
    location /static/ {
        alias /var/www/django_project/staticfiles/;
        expires 30d;
    }

    # 媒体文件配置
    location /media/ {
        alias /var/www/django_project/media/;
    }

    # 主应用配置
    location / {
        include uwsgi_params;
        uwsgi_pass 127.0.0.1:8000;
        uwsgi_read_timeout 300;
    }
}
```

##### 2. 启用配置并重启Nginx
```bash
# 启用配置
ln -s /etc/nginx/sites-available/django_project /etc/nginx/sites-enabled/

# 测试配置是否正确
nginx -t

# 重启Nginx
systemctl restart nginx

# 设置Nginx开机自启
systemctl enable nginx
```

#### 八、启动uWSGI服务

```bash
# 进入项目目录
cd /var/www/django_project

# 激活虚拟环境
workon django_env

# 启动uWSGI
uwsgi --ini uwsgi.ini
```

#### 九、配置uWSGI开机自启（可选）

##### 1. 创建系统服务文件
```bash
nano /etc/systemd/system/uwsgi.service
```

添加以下内容：
```ini
[Unit]
Description=uWSGI for Django Project
After=network.target

[Service]
User=root
Group=www-data
WorkingDirectory=/var/www/django_project
Environment="PATH=/root/.virtualenvs/django_env/bin"
ExecStart=/root/.virtualenvs/django_env/bin/uwsgi --ini uwsgi.ini
Restart=always

[Install]
WantedBy=multi-user.target
```

##### 2. 启用服务
```bash
systemctl daemon-reload
systemctl enable uwsgi
systemctl start uwsgi
```

#### 十、测试访问
在浏览器中输入`http://您的公网IP`，如果看到Django项目首页，说明部署成功！

#### 十一、常见问题排查

1. **无法访问网站**：检查安全组规则是否开放了80端口，Nginx和uWSGI是否正常运行
   ```bash
   # 检查Nginx状态
systemctl status nginx

   # 检查uWSGI状态
ps aux | grep uwsgi
   ```

2. **静态文件无法加载**：检查STATIC_ROOT配置和Nginx静态文件路径是否正确

3. **数据库连接错误**：确保db.sqlite3文件已上传到服务器，且权限正确
   ```bash
   # 修改文件权限
chmod 644 /var/www/django_project/db.sqlite3
   ```

希望这个详细步骤能帮助您成功部署Django网站！如有任何问题，请随时提问。
        